# Steps
[ ] Run inferences on a different dataset and use activations to calculate one-hot concept vector for each image  
[ ] Store prediction as label  
[ ] Train decision tree  
[ ] Get metrics for accuracy of decision tree predictions and actual alexnet predictions  

In [1]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torch import randperm
from torch._utils import _accumulate
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, sampler
from torch.utils.data.dataset import Subset
import matplotlib.pyplot as plt
import time
import os
import copy
from tqdm import tqdm, tqdm_notebook
import pandas as pd

import skimage

from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
from sklearn import tree

%load_ext autoreload
%autoreload 2

In [2]:
device = torch.device('cuda:0')


In [3]:
# def random_split(dataset, lengths):
#     """
#     Randomly split a dataset into non-overlapping new datasets of given lengths.

#     Arguments:
#         dataset (Dataset): Dataset to be split
#         lengths (sequence): lengths of splits to be produced
#     """
#     if sum(lengths) != len(dataset):
#         raise ValueError("Sum of input lengths does not equal the length of the input dataset!")

#     indices = randperm(sum(lengths))
#     return [Subset(dataset, indices[offset - length:offset]) for offset, length in zip(_accumulate(lengths), lengths)]

In [4]:
data_dir = 'data/cifar10'

model_name = 'alexnet'

num_classes = 8

batch_size = 20

num_epochs = 15

feature_extract = True

In [5]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [6]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [19]:
# initialize the model
model = models.alexnet(pretrained=True)
set_parameter_requires_grad(model, feature_extract)
num_ftrs = model.classifier[6].in_features

# change classifier layer
model.classifier[6] = nn.Linear(num_ftrs, num_classes)
input_size = 224

In [8]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

# # Create training and validation datasets
# image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
# # Create training and validation dataloaders


Initializing Datasets and Dataloaders...


In [9]:
train_data = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=data_transforms['train'])
train_loader = torch.utils.data.DataLoader(train_data, batch_size=4,
                                          shuffle=False, num_workers=2)

val_data = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=data_transforms['val'])
val_loader = torch.utils.data.DataLoader(val_data, batch_size=4,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [10]:
dataloaders = {
    'train': train_loader,
    'val': val_loader
}

In [20]:
# Send the model to GPU
model = model.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model.parameters()
print("Params to learn:")
params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)
        
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

RuntimeError: cuda runtime error (59) : device-side assert triggered at /opt/conda/conda-bld/pytorch_1525909934016/work/aten/src/THC/generic/THCTensorCopy.c:20

In [17]:
try:
    FileNotFoundError
except NameError:
    FileNotFoundError = IOError


In [18]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()
dataloaders_dict = {}
# Train and evaluate
model, hist = train_model(
        model, dataloaders, criterion, optimizer_ft, 
        num_epochs=num_epochs, is_inception=(model_name=="inception")
    )

RuntimeError: cuda runtime error (59) : device-side assert triggered at /opt/conda/conda-bld/pytorch_1525909934016/work/aten/src/THC/THCTensorCopy.cu:100

In [58]:
# max pool, flattens, then normalizes activations
def transform_activations(activations):
    m = nn.MaxPool2d(2)
    max_pool_outputs = m(activations)
    flattened_outputs = max_pool_outputs.reshape((max_pool_outputs.shape[0], np.prod(max_pool_outputs.shape[1:])))
    normalized_outputs = normalize(flattened_outputs)
    return normalized_outputs
    

In [59]:
# returns list of activations for a layer as well as the predicted class
def extract_activations(model, data_loader):
    activations = []
    classes = []
    
    activation_dict = {}
    
    # setup for evaluation mode
    model.eval()
    
    # define layer hook
    def layer_hook(module, input, output):
        layer_output = output.data.cpu()
        activations.append(layer_output)
    
    # register hook on layer
    handle = model.features[6].register_forward_hook(layer_hook) # register hook to access specific layer
    
    # do same thing for training set
    for inputs, labels in data_loader['train']:
        inputs = inputs.to(device)
        outputs = model(inputs).data.cpu()
        _, preds = torch.max(outputs, 1)

        classes.append(preds)

    activations = torch.cat(activations, dim=0)
    classes = torch.cat(classes, dim=0)
    
    activation_dict['train'] = (transform_activations(activations), classes)
    
    
    # reset and do the same for val set
    activations = []
    classes = []
    
    # do same thing for val set
    for inputs, labels in data_loader['val']:
        inputs = inputs.to(device)
        outputs = model(inputs).data.cpu()
        _, preds = torch.max(outputs, 1)

        classes.append(preds)
    
    activations = torch.cat(activations, dim=0)
    classes = torch.cat(classes, dim=0)
    
    activation_dict['val'] = (transform_activations(activations), classes)

    
    handle.remove()
    return activation_dict

In [61]:
model.eval()
activations = extract_activations(model, dataloaders)

test


In [62]:
activations['train'][0].shape

(5519, 13824)

In [63]:
def reduce_activations(activations):
    pca = PCA(n_components=100)
    pca.fit(activations)
    print("PCA score: %f" % np.sum(pca.explained_variance_ratio_))
    return pca
    

In [64]:
pca = reduce_activations(activations['train'][0])

PCA score: 0.572945


In [65]:
activations['train'][0].shape

(5519, 13824)

In [66]:
train_acts = pca.transform(normalize(activations['train'][0]))
val_acts = pca.transform(normalize(activations['val'][0]))
Y_train = activations['train'][1]
Y_val = activations['val'][1]

In [67]:
import pickle

# Load data (deserialize)
with open('classifiers.pickle', 'rb') as handle:
    classifiers = pickle.load(handle)

In [68]:
concepts = list(classifiers.keys())

In [85]:
def get_concept_vectors(activations, classifiers):
    predictions = []
    num_concepts = len(classifiers.keys())
    
    # for each classifier, get the prediction
    for clf in classifiers.items():
        pred = clf[1][0].predict(activations)[np.newaxis]
        pred = np.transpose(pred)
        predictions.append(pred)
    
    return np.hstack(predictions)

In [92]:
X_train = get_concept_vectors(train_acts, classifiers)
X_val = get_concept_vectors(val_acts, classifiers)

In [144]:
decision_tree = tree.DecisionTreeClassifier(min_samples_leaf=5, max_depth=10)
decision_tree = decision_tree.fit(X_train, Y_train)

In [145]:
decision_tree.score(X_val, Y_val)

0.8188405797101449

In [126]:

labels_frame = pd.read_csv('dataset/broden1_227/label.csv')

In [134]:
concept_names = labels_frame[labels_frame.number.isin(concepts)].name.tolist()
len(concept_names)

86

In [136]:
train_data.dataset.classes

['airplane', 'car', 'cat', 'dog', 'flower', 'fruit', 'motorbike', 'person']

In [ ]:
import graphviz


In [137]:
dot_data = tree.export_graphviz(decision_tree, out_file=None, feature_names=concept_names, class_names=train_data.dataset.classes)
graph = graphviz.Source(dot_data)
graph.render("trees/natural_images")

'trees/natural_images.pdf'

In [146]:
torch.cuda.empty_cache()